# Protein Translation and Feature Extraction Pipeline

This notebook performs end-to-end processing of variant-aligned FASTA files to generate model-ready feature matrices for phenotype prediction in *Mycobacterium tuberculosis*.

### Objectives:
- Load variant-aligned nucleotide sequences for a given resistance gene.
- Translate DNA sequences to protein sequences while handling gaps and frameshifts.
- Align translated sequences to the *H37Rv* reference protein.
- Annotate each sequence with resistance phenotype labels.
- Save both aligned amino acid sequences and a one-hot encoded feature matrix for machine learning models.

This pipeline supports downstream tasks including:
- Benchmarking classification performance across genes and drugs.
- Computing per-residue importance using classical and deep models.
- Linking sequence-level variation with known resistance mechanisms from WHO mutation catalogs.


## load required packages and files

In [1]:
import os, re, time, json, csv, joblib, sklearn
import numpy as np, pandas as pd
from Bio.Seq import Seq
from Bio import SeqIO
from evcouplings.compare import DistanceMap
from Bio.Data import CodonTable
from joblib import Parallel, delayed

In [2]:
from data_loading import *
from data_preprocessing import *
from sequence_translation import *

## initialize by gene name

In [3]:
gene_name='rpsL'
genes_of_interest=gene_name.split(',') #useful if running for multiple proteins
# Set random seed for reproducibility
seed_everything(42)

## Required data files

In [4]:
# Load phenotype data and gene metadata
phenotype_paths = ["data/master_table_resistance.csv","data/cryptic_dataset.csv"]  # resistance labels
gene_details = pd.read_csv("data/all_drug_genes_locations.csv") # details on genomic accessions and drugs
fasta_dir = "data/cryptic_fasta_files"

In [5]:
# Filter the DataFrame for target gene
filtered_df = gene_details [gene_details ['gene_name'].str.contains('|'.join(genes_of_interest), case=False, na=False)]
drug_name = filtered_df['drug_full'].values[0].upper()
uniprot = filtered_df['Uniprot'].values[0]
entry = filtered_df['Entry'].values[0]
print(f"The drug associated with the gene {gene_name} is: {drug_name}")

The drug associated with the gene rpsL is: STREPTOMYCIN


In [6]:
#load and clean phenotype data
phenotype_data = load_phenotype_data(phenotype_paths,drug_name).reset_index()
phenotype_data=phenotype_data.drop(['level_0'], axis=1)
phenotype_data["Isolate_mapped"] = [path.split("/")[-1].split(".vcf")[0].split(".cut")[0] for path in phenotype_data.path]
phenotype_mapping = dict(zip(phenotype_data['Isolate_mapped'], phenotype_data[drug_name.upper()]))

Number of records in phenotype data: 17545


In [8]:
# Process each FASTA alignment and extract the gene region
subset_alignment=''
h37rv_nongap_indices=[]
for index, row in filtered_df.iterrows():
    # Extract gene-related info from metadata
    fasta_filename = row['filename']
    print(f"Processing file: {fasta_filename}")
    start_index = row['start_position_on_the_genomic_accession']
    end_index = row['end_position_on_the_genomic_accession']
    uniprot = row['Uniprot']
    entry = row['Entry']
    drug = row['drug_full'] 
    drug_code = row['Drug']
    gene = row['gene_name']
    orientation = row['orientation']
    file_path = os.path.join(fasta_dir, fasta_filename)
    filename = os.path.basename(file_path)


    # Load and inspect alignment matrix
    alignment = load_alignment(file_path)
    # print(f"Loaded alignment matrix for {filename}")
    print(f"Gene: {gene}, Drug: {drug}")
    print(f"gene shape: {alignment.matrix.shape}")

    # Load H37Rv reference column-wise coordinates
    h37rv_numbers = np.load("data/X_matrix_H37RV_coords.npy")
    h37rv_ref = alignment.select(sequences=[alignment.id_to_index["MT_H37Rv"]])
    # Extract gene-specific slice of the alignment
    subset_alignment, column_index, start_index, end_index = sort_gene_indices(h37rv_numbers, start_index, end_index, alignment)
    if column_index is not None:
        print(f"Using column {column_index} for alignment selection.")
        print(f"Reference numbers for this column: {h37rv_numbers[:, column_index]}")
    else:
        raise ValueError(f" No valid column found for gene start {start_index}.")

    h37rv_alignment = select_subset_alignment(h37rv_ref, start_index, end_index,h37rv_numbers[:, column_index])
    # Extract gene-specific slice of the alignment
    h37rv_nongap_indices = np.where(h37rv_alignment[0] != "-")[0]
    h37rv_sequence_str = ''.join(h37rv_alignment[0][h37rv_nongap_indices])

Processing file: rpsL.fasta
Gene: rpsL, Drug: streptomycin
gene shape: (31452, 651)
Processed subset alignment for gene start 781559.0 and end 781933.0 in column 11
Using column 11 for alignment selection.
Reference numbers for this column: [781311. 781312. 781313. ...      0.      0.      0.]


In [9]:
# Convert alignment to dictionary mapping filenames to sequences
filenames = [path.split("/")[-1].split(".cut")[0] for path in subset_alignment.ids]
filename_to_sequence = {}
for i, filename in enumerate(filenames):
    if filename not in filename_to_sequence and i < len(subset_alignment):
        filename_to_sequence[filename] = ''.join(subset_alignment[i])
filenames = list(filename_to_sequence.keys()) 

## Data preparation: from DNA seq to protein translation

In [11]:
# Define the output file path
output_file = f"data/sequence_data_csv/{gene_name}_sequence_data.csv"


In [12]:
# Write CSV with per-sample nucleotide sequences and phenotypes
data_list = []
for i in range(len(filenames)):
    filename = filenames[i]

    if filename in phenotype_mapping and filename in filename_to_sequence:
        # Fetch the corresponding phenotype
        phenotype = phenotype_mapping[filename]
        # Fetch the corresponding sequence
        sequence = filename_to_sequence[filename]
        ## we include the nongap indices here so that we're only looking at mutations relative to h37rv
        sequence = "".join(np.array(list(sequence))[h37rv_nongap_indices])
        # Append to list
        data_list.append([filename, sequence, phenotype, len(sequence)])

# Save data to CSV
with open(output_file, mode="w", newline="") as file:
    writer = csv.writer(file)
    writer.writerow(["Filename", "Sequence", "Phenotype","seq_len"])  # Header
    writer.writerows(data_list)

print(f"CSV file '{output_file}' saved successfully!")

CSV file 'data/sequence_data_csv/rpsL_sequence_data.csv' saved successfully!


## Convert nucleotide sequence to protein sequence

In [14]:
gene_sequence_data=pd.read_csv(output_file)

In [15]:
# Translate each DNA sequence to amino acids and handle orientation/frameshifts
all_translations = []           # Stores final aligned amino acid sequences
all_labels = []                 # Stores phenotype labels
problematic_indices = set()    # Indices where frameshifts were detected
frameshift_mutations_list = [] # Binary flag: 1 if frameshift, 0 otherwise

# Loop through all sequences for the current gene
for i in range(len(gene_sequence_data)):
    problematic = False  # Track if the current sequence has issues
    
    # Determine translation based on strand orientation
    if orientation == 'plus':
        h37rv_aa_str = Seq(h37rv_sequence_str).translate()  # Translate H37Rv reference for '+' strand
        reference_length = len(h37rv_aa_str)
        translation, frameshift = translate_sequence_with_gaps(
            gene_sequence_data["Sequence"][i],
            to_stop=False,
            ref_protein_length=reference_length
        )
    else:
        # Reverse complement for '-' strand before translation
        h37rv_aa_str = Seq(h37rv_sequence_str).reverse_complement().translate()
        reference_length = len(h37rv_aa_str)
        template_dna = Seq(gene_sequence_data["Sequence"][i]).reverse_complement()
        translation, frameshift = translate_sequence_with_gaps(
            str(template_dna),
            to_stop=False,
            ref_protein_length=reference_length
        )

    # Align the translated sequence to the reference (handles insertions/deletions)
    aligned_translation = align_and_handle_deletions(translation, h37rv_aa_str)

    # Print debug information for the second sequence (optional)
    if i == 1:
        print("translation before aligning", translation)
        print("translation after aligning", aligned_translation)
        print(f"Translated length: {len(translation)}, Reference length: {reference_length}")
        print(f"Final aligned sequence: {aligned_translation}")

    # Handle detected frameshifts by marking and replacing the sequence with 0s
    if frameshift:
        problematic = True
        problematic_indices.add(i)
        frameshift_mutations_list.append(1)
        aligned_translation = '0' * reference_length  # Placeholder for unusable sequences
    else:
        frameshift_mutations_list.append(0)

    # Store aligned sequence and phenotype label
    all_translations.append(aligned_translation)
    all_labels.append(gene_sequence_data["Phenotype"][i])

# Add aligned amino acid sequences and frameshift mutation flags to DataFrame
gene_sequence_data["Protein_Sequence"] = all_translations
gene_sequence_data["Frameshift_Mutation"] = frameshift_mutations_list


/work/pi_annagreen_umass_edu/mahbuba/protein/lib/python3.10/site-packages/Bio/Seq.py:2804: BiopythonWarning: Partial codon, len(sequence) not a multiple of three. Explicitly trim the sequence or add trailing N before translation. This may become an error in future.
  warnings.warn(


translation before aligning MPTIQQLVRKGRRDKISKVKTAALKGSPQRRGVCTRVYTTTPRKPNSALRKVARVKLTSQVEVTAYIPGEGHNLQEHSMVLVRGGRVKDLPGVRYKIIRGSLDTQGVKNRKQARSRYGAKKEKG
translation after aligning MPTIQQLVRKGRRDKISKVKTAALKGSPQRRGVCTRVYTTTPRKPNSALRKVARVKLTSQVEVTAYIPGEGHNLQEHSMVLVRGGRVKDLPGVRYKIIRGSLDTQGVKNRKQARSRYGAKKEKG
Translated length: 124, Reference length: 124
Final aligned sequence: MPTIQQLVRKGRRDKISKVKTAALKGSPQRRGVCTRVYTTTPRKPNSALRKVARVKLTSQVEVTAYIPGEGHNLQEHSMVLVRGGRVKDLPGVRYKIIRGSLDTQGVKNRKQARSRYGAKKEKG


## add protein sequence to the gene csv file and save

In [16]:
gene_sequence_data.to_csv(output_file, index=False)

In [17]:
# --- Diagnostic stats on frameshifts and alignment success ---

# Count the number of sequences with and without frameshifts
unique_elements, counts = np.unique(frameshift_mutations_list, return_counts=True)
value_counts_dict = dict(zip(unique_elements, counts))  # e.g., {0: valid_count, 1: frameshift_count}
print(value_counts_dict)

# Calculate the percentage of sequences flagged as problematic due to frameshift mutations
problem_percentage = (len(problematic_indices) / len(gene_sequence_data)) * 100
print(f"Problematic percentage: {problem_percentage}%")

# Proceed with keeping all sequences, regardless of frameshift status (for interpretability, etc.)
print("Keeping all sequences.")
valid_indices = range(len(gene_sequence_data))

# Truncate all aligned translations to the reference length (for consistency)
valid_translations = [(filenames[i], all_translations[i][:reference_length]) for i in valid_indices]

# Corresponding phenotype labels
valid_labels = [all_labels[i] for i in valid_indices]

# Check sequence lengths after truncation
valid_lengths = [len(seq[1]) for seq in valid_translations]

# Identify any sequences that still deviate from the expected length
length_mismatches = [i for i, length in enumerate(valid_lengths) if length != reference_length]
total_sequences = len(valid_translations)
num_mismatches = len(length_mismatches)

# Display summary of sequence lengths
print(f"Reference length expected: {reference_length}")
print(f"Sample of sequence lengths after truncation: {[len(seq[1]) for seq in valid_translations[:10]]}")

# Report whether sequence lengths are consistent
if num_mismatches == 0:
    print("All translations have the correct length.")
else:
    mismatch_percentage = (num_mismatches / total_sequences) * 100
    print(f"{num_mismatches} sequences have varying lengths out of {total_sequences} sequences.")
    print(f"Percentage of sequences with varying lengths: {mismatch_percentage:.2f}%")
    # Optional debug info:
    # print(f"Indices of sequences with varying lengths: {length_mismatches}")
    # print(f"Lengths of the mismatched sequences: {[valid_lengths[i] for i in length_mismatches]}")

# Final stats
print(f"Total sequences after filtering: {len(valid_translations)}")
print(f"Sample of sequence lengths after filtering: {[len(seq[1]) for seq in valid_translations[:10]]}")


{0: 17542}
Problematic percentage: 0.0%
Keeping all sequences.
Reference length expected: 124
Sample of sequence lengths after truncation: [124, 124, 124, 124, 124, 124, 124, 124, 124, 124]
All translations have the correct length.
Total sequences after filtering: 17542
Sample of sequence lengths after filtering: [124, 124, 124, 124, 124, 124, 124, 124, 124, 124]


### write aa fasta file

In [18]:
# Write full translated sequences in FASTA format
protein_name = f'data/full_sequence_alignments/all_sequences_{gene_name}_aa.fasta'

# Ensure the DataFrame has the correct columns
if {"Filename", "Protein_Sequence", "Phenotype", "Frameshift_Mutation"}.issubset(gene_sequence_data.columns):
    write_fasta_with_metadata_from_df(gene_sequence_data, protein_name, reference_length)
    print(f"FASTA file saved at {protein_name}")
else:
    print("Error: The DataFrame is missing required columns.")


FASTA file saved at data/full_sequence_alignments/all_sequences_rpsL_aa.fasta


### save feature matrix and feature labels

In [19]:
# Encode protein sequences relative to H37Rv reference and generate feature matrix
num_cores = joblib.cpu_count()

encoded_features = Parallel(n_jobs=num_cores)(
    delayed(encode_sequence)(valid_translations[i][1], reference_length, h37rv_aa_str) for i in range(len(valid_translations)))


mutation_flags = np.array(frameshift_mutations_list).reshape(-1, 1)

feature_matrix = np.column_stack((mutation_flags, np.array(encoded_features)))

print(f"Feature matrix shape: {feature_matrix.shape}")

# Save feature matrix and corresponding labels
np.save(f'data/feature_matrix_labels/{gene_name}_feature_matrix.npy', feature_matrix)
np.save(f'data/feature_matrix_labels/{gene_name}_labels.npy', valid_labels)

Feature matrix shape: (17542, 125)
